In [1]:
import pandas as pd
import re
import string
import nltk
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words('portuguese')
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import json
import time
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alici\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
pd.options.mode.chained_assignment = None  # default='warn'

In [3]:
file_path = r"C:\Users\alici\Documents\UFLA\mestrado\projeto\dicionario\2024\brMoral\brMoral.csv"

In [4]:
def read_file_as_df(file_name):
    import pandas as pd
    import csv

    import sys
    import pandas as pd

    maxInt = sys.maxsize

    while True:
        # decrease the maxInt value by factor 10
        # as long as the OverflowError occurs.

        try:
            csv.field_size_limit(maxInt)
            break
        except OverflowError:
            maxInt = int(maxInt/10)

    file = []
    col = []

    with open(file_name, encoding='latin-1') as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=';')
        line_count = 0
        for row in csv_reader:
            if line_count==0:
                for r in row:
                    col.append(r)
                line_count+=1
            else:
                line = []
                for r in row:
                    line.append(r)
                file.append(line)
                line_count += 1


    df = pd.DataFrame(file, columns = col)
    return df

In [5]:
def remove_stopwords(text):
    all_words = text.split(" ")
    clean_text = [i for i in all_words if i not in stopwords and i!=""]
    return " ".join(clean_text)

In [6]:
def clean_text(x):
  return " ".join(str(x.translate(str.maketrans('', '', string.punctuation))).split()).lower()

In [7]:
def select_abs(col, perc):
  n = int(len(df_coef) * perc)

  df_coef_1 = df_coef.copy()
  df_coef_1 = df_coef_1[[col]]

  df_coef_1['abs_values'] = df_coef_1[col].abs()

  sorted_df = df_coef_1.sort_values(by='abs_values', ascending=False)

  return sorted_df.head(n)

In [8]:
fold_path = r'C:\Users\alici\Documents\UFLA\mestrado\projeto\dicionario\2024\brMoral\folds'

In [9]:
with open(r"C:\Users\alici\Documents\UFLA\mestrado\projeto\dicionario\2024\brMoral\age.json", encoding='utf-8') as f:
    age_param = json.load(f)

In [10]:
age_param

{'grid_values': {'C': [0.001, 0.01, 0.1, 1.0, 10.0, 100.0, 1000.0, 2857.14],
  'penalty': ['l1', 'l2']},
 'k_1': {'params': {'C': 10.0, 'penalty': 'l1'}, 'f1': 0.44333333333333336},
 'k_2': {'params': {'C': 1.0, 'penalty': 'l2'}, 'f1': 0.4700000000000001},
 'k_3': {'params': {'C': 1.0, 'penalty': 'l2'}, 'f1': 0.6066666666666666},
 'k_4': {'params': {'C': 10.0, 'penalty': 'l2'}, 'f1': 0.4833333333333334},
 'k_5': {'params': {'C': 1000.0, 'penalty': 'l1'}, 'f1': 0.5466666666666666},
 'k_6': {'params': {'C': 0.001, 'penalty': 'l1'}, 'f1': 0.3733333333333334},
 'k_7': {'params': {'C': 1.0, 'penalty': 'l2'}, 'f1': 0.4566666666666667},
 'k_8': {'params': {'C': 0.001, 'penalty': 'l1'}, 'f1': 0.3533333333333334},
 'k_9': {'params': {'C': 1000.0, 'penalty': 'l2'}, 'f1': 0.41},
 'k_10': {'params': {'C': 100.0, 'penalty': 'l1'}, 'f1': 0.38666666666666666},
 'time': '0.0min8.49s'}

In [11]:
train_df =  pd.read_csv(fold_path+r'\k_'+str(1)+r'\train.csv')
test_df =  pd.read_csv(fold_path+r'\k_'+str(1)+r'\test.csv')

In [12]:
df_merged = pd.concat([train_df, test_df], ignore_index=True, sort=False)

In [13]:
df_merged['num_palavras'] = df_merged['concat'].apply(lambda x: len(str(x).split()))

In [14]:
df_merged.head()

,sid,concat,freetext,gender,ap.age,num_palavras
0,sid6000,"Não vejo e nem nunca vi problema nisso, acho q...",A flexibilidade é extremamente importante nas ...,m,1,272
1,sid6001,Acredito que todos possuem direito de escolher...,Acabo de responder a perguntas bastante polêmi...,m,2,416
2,sid959,Acredito que todos tem o direito de escolher c...,"Achei esse teste cansativo à beça, mas me fez ...",m,2,373
3,sid842,Vejo que as pessoas tem todo o direito de esco...,amanhã trabalho cansaço tarde engenharia dados...,m,1,664
4,sid774,As pessoas devem ser livres para tomar decisõe...,Algo difícil na vida é conquistar sua independ...,f,1,304


In [15]:
df_merged['num_palavras'].mean()

427.8137254901961

In [10]:
for i in range(1, 11):
  train_df =  pd.read_csv(fold_path+r'\k_'+str(i)+r'\train.csv')
  test_df =  pd.read_csv(fold_path+r'\k_'+str(i)+r'\test.csv')
  train_df = train_df[['sid', 'concat', 'ap.age']]
  test_df = test_df[['sid', 'concat', 'ap.age']]

  train_df["Clean_text"] = train_df.concat.apply(lambda x: clean_text(x))
  train_df["Clean_text"] = train_df.Clean_text.apply(lambda x: remove_stopwords(x))

  test_df["Clean_text"] = test_df.concat.apply(lambda x: clean_text(x))
  test_df["Clean_text"] = test_df.Clean_text.apply(lambda x: remove_stopwords(x))

  unique_words = list(set(" ".join(train_df.Clean_text.to_list()).split()))

  threshold = train_df.shape[0]*0.01
  word_counts = train_df['Clean_text'].str.split().explode().value_counts()
  row_counts = pd.DataFrame({'word': word_counts.index, 'count': word_counts.values})
  row_counts['count'] = row_counts['word'].apply(lambda word: train_df['Clean_text'].str.contains(word).sum())
  selected_words = row_counts[row_counts['count']>threshold]['word'].to_list()
  print(len(selected_words))

3761
3815
3767
3787
3769
3787
3733
3758
3790
3790


In [16]:
def select_(col, perc): # mesma quantidade de palavras positivas e negativas
  df_coef_1 = df_coef.copy()
  df_coef_1 = df_coef_1[[col]]

  str_col = str(col)

  df_coef_1.rename(columns={col: str_col}, inplace=True)

  df_coef_1_pos = df_coef_1[df_coef_1[str_col]>0]
  df_coef_1_neg= df_coef_1[df_coef_1[str_col]<0]
  df_coef_1_zero = df_coef_1[df_coef_1[str_col]==0]

  df_coef_1_pos = df_coef_1_pos.sort_values(by=str_col, ascending=True)
  df_coef_1_neg = df_coef_1_neg.sort_values(by=str_col, key=lambda x: abs(x))

  pos_perc = df_coef_1_pos.head(int(len(df_coef_1_pos) * perc))
  neg_perc = df_coef_1_neg.head(int(len(df_coef_1_neg) * perc))

  indices_to_drop = pd.concat([pos_perc, neg_perc, df_coef_1_zero]).index

  df_filtered = df_coef_1.drop(indices_to_drop)

  return df_filtered.sort_values(by=str_col, ascending=False)

In [12]:
train_df =  pd.read_csv(fold_path+r'\k_'+str(1)+r'\train.csv')
test_df =  pd.read_csv(fold_path+r'\k_'+str(1)+r'\test.csv')
train_df = train_df[['sid', 'concat', 'ap.age']]
test_df = test_df[['sid', 'concat', 'ap.age']]

train_df["Clean_text"] = train_df.concat.apply(lambda x: clean_text(x))
train_df["Clean_text"] = train_df.Clean_text.apply(lambda x: remove_stopwords(x))

test_df["Clean_text"] = test_df.concat.apply(lambda x: clean_text(x))
test_df["Clean_text"] = test_df.Clean_text.apply(lambda x: remove_stopwords(x))

unique_words = list(set(" ".join(train_df.Clean_text.to_list()).split()))

threshold = train_df.shape[0]*0.01
word_counts = train_df['Clean_text'].str.split().explode().value_counts()
row_counts = pd.DataFrame({'word': word_counts.index, 'count': word_counts.values})
row_counts['count'] = row_counts['word'].apply(lambda word: train_df['Clean_text'].str.contains(word).sum())
selected_words = row_counts[row_counts['count']>threshold]['word'].to_list()

text = train_df.Clean_text.to_list()
vectorizer = TfidfVectorizer(vocabulary=selected_words)
train_matrix = vectorizer.fit_transform(text)
train_matrix = train_matrix.toarray()
vocab = vectorizer.get_feature_names_out()

tf_idf = pd.DataFrame(data=train_matrix, columns=selected_words)

X = tf_idf
y =  train_df['ap.age']

In [39]:
X

,pessoas,acredito,deve,sociedade,ter,vida,deveria,forma,aborto,morte,...,manipula,ataca,sul,ocasiona,submete,tidos,festa,conhecer,sincera,mass
0,0.027904,0.081629,0.000000,0.105703,0.033114,0.034667,0.000000,0.072647,0.033647,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.062944,0.061379,0.026804,0.026494,0.049799,0.078202,0.000000,0.081938,0.025300,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.239088,0.059663,0.088457,0.000000,0.029011,0.094050,0.000000,0.028157,0.080002,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.131206,0.023989,0.020952,0.062129,0.019463,0.061129,0.000000,0.021350,0.059329,0.018730,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.049195,0.143916,0.000000,0.062120,0.058382,0.061120,0.000000,0.064040,0.029661,0.056182,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
454,0.000000,0.000000,0.153615,0.000000,0.057080,0.089634,0.000000,0.062611,0.028999,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
455,0.000000,0.054648,0.047730,0.023588,0.000000,0.046417,0.050159,0.000000,0.045051,0.021334,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
456,0.113409,0.000000,0.000000,0.000000,0.000000,0.000000,0.152258,0.036907,0.000000,0.032379,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
457,0.090504,0.165475,0.000000,0.028570,0.000000,0.028110,0.151883,0.000000,0.027283,0.051679,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
for i in range(1, 11):
  train_df =  pd.read_csv(fold_path+r'\k_'+str(i)+r'\train.csv')
  test_df =  pd.read_csv(fold_path+r'\k_'+str(i)+r'\test.csv')
  train_df = train_df[['sid', 'concat', 'ap.age']]
  test_df = test_df[['sid', 'concat', 'ap.age']]

  train_df["Clean_text"] = train_df.concat.apply(lambda x: clean_text(x))
  train_df["Clean_text"] = train_df.Clean_text.apply(lambda x: remove_stopwords(x))

  test_df["Clean_text"] = test_df.concat.apply(lambda x: clean_text(x))
  test_df["Clean_text"] = test_df.Clean_text.apply(lambda x: remove_stopwords(x))

  unique_words = list(set(" ".join(train_df.Clean_text.to_list()).split()))

  threshold = train_df.shape[0]*0.01
  word_counts = train_df['Clean_text'].str.split().explode().value_counts()
  row_counts = pd.DataFrame({'word': word_counts.index, 'count': word_counts.values})
  row_counts['count'] = row_counts['word'].apply(lambda word: train_df['Clean_text'].str.contains(word).sum())
  selected_words = row_counts[row_counts['count']>threshold]['word'].to_list()

  text = train_df.Clean_text.to_list()
  vectorizer = TfidfVectorizer(vocabulary=selected_words)
  train_matrix = vectorizer.fit_transform(text)
  train_matrix = train_matrix.toarray()
  vocab = vectorizer.get_feature_names_out()

  tf_idf = pd.DataFrame(data=train_matrix, columns=selected_words)

  X = tf_idf
  y =  train_df['ap.age']

  svc = SVC(kernel='linear', decision_function_shape='ovr')
  svc.fit(X, y)

  df_coef = pd.DataFrame(data=svc.coef_, columns=tf_idf.columns)
  df_coef = df_coef.T

  coef1 = select_abs(0, 1)
  coef2 = select_abs(1, 1)
  coef3 = select_abs(2, 1)

  train_df['glex1']=0
  train_df['glex2']=0
  train_df['glex3']=0

  train_df.reset_index(inplace=True)

  for index, row in train_df.iterrows():
      sum = 0
      text = row["Clean_text"].split()
      for w in text:
        if w in coef1.index.tolist():
          sum += coef1[0][w]*tf_idf[w][index]
      sum = sum + svc.intercept_[0]
      train_df[f'glex1'][index] = sum

  for index, row in train_df.iterrows():
      sum = 0
      text = row["Clean_text"].split()
      for w in text:
        if w in coef2.index.tolist():
          sum += coef2[1][w]*tf_idf[w][index]
      sum = sum + svc.intercept_[1]
      train_df[f'glex2'][index] = sum

  for index, row in train_df.iterrows():
      sum = 0
      text = row["Clean_text"].split()
      for w in text:
        if w in coef3.index.tolist():
          sum += coef3[2][w]*tf_idf[w][index]
      sum = sum + svc.intercept_[2]
      train_df[f'glex3'][index] = sum

  train_df['c'] = 0

  for index, row in train_df.iterrows():
    if row['glex1'] > 0 and row['glex2'] > 0 and row['glex3'] > 0:
      train_df['c'][index] = 1
    elif row['glex1'] < 0 and row['glex2'] > 0 and row['glex3'] > 0:
      train_df['c'][index] = 2
    elif row['glex1'] < 0 and row['glex2'] < 0 and row['glex3'] < 0:
      train_df['c'][index] = 3

  train_df['ap.age'] = train_df['ap.age'].astype('int64')

  # print(f'Porcentagem classificadas: {train_df[train_df.c!=0].shape[0]/train_df.shape[0]}')
  # print(f'Porcentagem classificadas corretamente: {train_df[(train_df.c!=0) & (train_df.c==train_df["ap.age"])].shape[0]/train_df[train_df.c!=0].shape[0]}')
  # print(f'Porcentagem classificadas corretamente das totais: {train_df[(train_df.c!=0) & (train_df.c==train_df["ap.age"])].shape[0]/train_df.shape[0]}')

  test_matrix = vectorizer.transform(test_df["Clean_text"].to_list())
  test_matrix = test_matrix.toarray()

  tf_idf_test = pd.DataFrame(data=test_matrix, columns=vocab)

  test_df.reset_index(inplace=True)

  test_df['glex1']=0
  test_df['glex2']=0
  test_df['glex3']=0

  for index, row in test_df.iterrows():
      sum = 0
      text = row["Clean_text"].split()
      for w in text:
        if w in coef1.index.tolist():
          sum += coef1[0][w]*tf_idf_test[w][index]
      sum = sum + svc.intercept_[0]
      test_df[f'glex1'][index] = sum

  for index, row in test_df.iterrows():
      sum = 0
      text = row["Clean_text"].split()
      for w in text:
        if w in coef2.index.tolist():
          sum += coef2[1][w]*tf_idf_test[w][index]
      sum = sum + svc.intercept_[1]
      test_df[f'glex2'][index] = sum

  for index, row in test_df.iterrows():
      sum = 0
      text = row["Clean_text"].split()
      for w in text:
        if w in coef3.index.tolist():
          sum += coef3[2][w]*tf_idf_test[w][index]
      sum = sum + svc.intercept_[2]
      test_df[f'glex3'][index] = sum

  test_df['c'] = 0

  for index, row in test_df.iterrows():
    if row['glex1'] > 0 and row['glex2'] > 0 and row['glex3'] > 0:
      test_df['c'][index] = 1
    elif row['glex1'] < 0 and row['glex2'] > 0 and row['glex3'] > 0:
      test_df['c'][index] = 2
    elif row['glex1'] < 0 and row['glex2'] < 0 and row['glex3'] < 0:
      test_df['c'][index] = 3

  test_df['ap.age'] = test_df['ap.age'].astype('int64')

  print(f"Classificação por dicionário: {test_df[test_df.c!=0].shape[0]} instâncias")
  print(f'Porcentagem classificadas: {test_df[test_df.c!=0].shape[0]/test_df.shape[0]}')
  print(f'Classificadas corretamente: {test_df[(test_df.c!=0) & (test_df.c==test_df["ap.age"])].shape[0]/test_df[test_df.c!=0].shape[0]}')
  
  print(f"Classificação tradicional: {test_df[test_df.c==0].shape[0]} instâncias")
  
  test_df_1 =  test_df[test_df.c==0]

  tfvec = TfidfVectorizer(max_features = 3000)
  tfvec.fit(train_df['Clean_text'].to_list())
  tdf_train = tfvec.transform(train_df['Clean_text'].to_list()).toarray().tolist()
  tdf_test = tfvec.transform(test_df_1['Clean_text'].to_list()).toarray().tolist()

  age_train = train_df["ap.age"].to_list()
  age_test = test_df_1["ap.age"].to_list()

  logisticRegr = LogisticRegression(penalty=age_param["k_"+str(i)]["params"]["penalty"], C=age_param["k_"+str(i)]["params"]["C"], solver='liblinear')
  logisticRegr.fit(tdf_train, age_train)
  pred_age=logisticRegr.predict(tdf_test)
  #print(f"F1: {metrics.f1_score(age_test, pred_age, average='macro')}")
  
  dict_class = test_df[test_df.c!=0]['ap.age']
  dict_pred = test_df[test_df.c!=0]['c']

  class_total = age_test+dict_class.to_list()
  pred_total = pred_age.tolist()+dict_pred.to_list()

  print(f"RESULTADO: {metrics.f1_score(class_total, pred_total, average='macro')}")
  print("-----------------------------------------------------------------------")

Classificação por dicionário: 38 instâncias
Porcentagem classificadas: 0.7450980392156863
Classificadas corretamente: 0.5
Classificação tradicional: 13 instâncias
RESULTADO: 0.48081081081081084
-----------------------------------------------------------------------
Classificação por dicionário: 45 instâncias
Porcentagem classificadas: 0.8823529411764706
Classificadas corretamente: 0.5555555555555556
Classificação tradicional: 6 instâncias
RESULTADO: 0.5073610599926389
-----------------------------------------------------------------------
Classificação por dicionário: 41 instâncias
Porcentagem classificadas: 0.803921568627451
Classificadas corretamente: 0.6341463414634146
Classificação tradicional: 10 instâncias
RESULTADO: 0.5679671205986995
-----------------------------------------------------------------------
Classificação por dicionário: 38 instâncias
Porcentagem classificadas: 0.7450980392156863
Classificadas corretamente: 0.5263157894736842
Classificação tradicional: 13 instância

In [18]:
with open(r'C:\Users\alici\Documents\UFLA\mestrado\projeto\dicionario\2024\param_289_1.json', encoding='utf-8') as f:
    dict_param = json.load(f)

In [29]:
dict_param

{'grid_values': {'C': [0.1, 1, 100],
  'tol': [0.0001, 0.01],
  'max_iter': [1000, -1]},
 'k_2': {'params': {'C': 1, 'max_iter': 1000, 'tol': 0.0001},
  'accuracy': 0.4942493521828578},
 'k_3': {'params': {'C': 1, 'max_iter': 1000, 'tol': 0.0001},
  'accuracy': 0.5113877055193046},
 'k_4': {'params': {'C': 100, 'max_iter': 1000, 'tol': 0.0001},
  'accuracy': 0.46401666920941836},
 'k_5': {'params': {'C': 100, 'max_iter': 1000, 'tol': 0.0001},
  'accuracy': 0.4848739398198131},
 'k_6': {'params': {'C': 1, 'max_iter': 1000, 'tol': 0.0001},
  'accuracy': 0.49088131030976634},
 'k_7': {'params': {'C': 1, 'max_iter': 1000, 'tol': 0.0001},
  'accuracy': 0.48728239876892576},
 'k_8': {'params': {'C': 100, 'max_iter': 1000, 'tol': 0.0001},
  'accuracy': 0.4764956996401838},
 'k_9': {'params': {'C': 100, 'max_iter': 1000, 'tol': 0.0001},
  'accuracy': 0.4838233617005646},
 'k_10': {'params': {'C': 100, 'max_iter': 1000, 'tol': 0.0001},
  'accuracy': 0.4856383821026232},
 'k_11': {'params': {'C'

In [ ]:
for i in range(1, 11):
  train_df =  pd.read_csv(fold_path+r'\k_'+str(i)+r'\train.csv')
  test_df =  pd.read_csv(fold_path+r'\k_'+str(i)+r'\test.csv')
  train_df = train_df[['sid', 'concat', 'ap.age']]
  test_df = test_df[['sid', 'concat', 'ap.age']]

  train_df["Clean_text"] = train_df.concat.apply(lambda x: clean_text(x))
  train_df["Clean_text"] = train_df.Clean_text.apply(lambda x: remove_stopwords(x))

  test_df["Clean_text"] = test_df.concat.apply(lambda x: clean_text(x))
  test_df["Clean_text"] = test_df.Clean_text.apply(lambda x: remove_stopwords(x))

  unique_words = list(set(" ".join(train_df.Clean_text.to_list()).split()))

  threshold = train_df.shape[0]*0.01
  word_counts = train_df['Clean_text'].str.split().explode().value_counts()
  row_counts = pd.DataFrame({'word': word_counts.index, 'count': word_counts.values})
  row_counts['count'] = row_counts['word'].apply(lambda word: train_df['Clean_text'].str.contains(word).sum())
  selected_words = row_counts[row_counts['count']>threshold]['word'].to_list()

  text = train_df.Clean_text.to_list()
  vectorizer = TfidfVectorizer(vocabulary=selected_words)
  train_matrix = vectorizer.fit_transform(text)
  train_matrix = train_matrix.toarray()
  vocab = vectorizer.get_feature_names_out()

  tf_idf = pd.DataFrame(data=train_matrix, columns=selected_words)

  X = tf_idf
  y =  train_df['ap.age']

  svc = SVC(kernel='linear', decision_function_shape='ovr')
  svc.fit(X, y)

  df_coef = pd.DataFrame(data=svc.coef_, columns=tf_idf.columns)
  df_coef = df_coef.T

  coef1 = select_abs(0, 1)
  coef2 = select_abs(1, 1)
  coef3 = select_abs(2, 1)

  train_df['glex1']=0
  train_df['glex2']=0
  train_df['glex3']=0

  train_df.reset_index(inplace=True)

  for index, row in train_df.iterrows():
      sum = 0
      text = row["Clean_text"].split()
      for w in text:
        if w in coef1.index.tolist():
          sum += coef1[0][w]*tf_idf[w][index]
      sum = sum + svc.intercept_[0]
      train_df[f'glex1'][index] = sum

  for index, row in train_df.iterrows():
      sum = 0
      text = row["Clean_text"].split()
      for w in text:
        if w in coef2.index.tolist():
          sum += coef2[1][w]*tf_idf[w][index]
      sum = sum + svc.intercept_[1]
      train_df[f'glex2'][index] = sum

  for index, row in train_df.iterrows():
      sum = 0
      text = row["Clean_text"].split()
      for w in text:
        if w in coef3.index.tolist():
          sum += coef3[2][w]*tf_idf[w][index]
      sum = sum + svc.intercept_[2]
      train_df[f'glex3'][index] = sum

  train_df['c'] = 0

  for index, row in train_df.iterrows():
    if row['glex1'] > 0 and row['glex2'] > 0 and row['glex3'] > 0:
      train_df['c'][index] = 1
    elif row['glex1'] < 0 and row['glex2'] > 0 and row['glex3'] > 0:
      train_df['c'][index] = 2
    elif row['glex1'] < 0 and row['glex2'] < 0 and row['glex3'] < 0:
      train_df['c'][index] = 3

  train_df['ap.age'] = train_df['ap.age'].astype('int64')

  # print(f'Porcentagem classificadas: {train_df[train_df.c!=0].shape[0]/train_df.shape[0]}')
  # print(f'Porcentagem classificadas corretamente: {train_df[(train_df.c!=0) & (train_df.c==train_df["ap.age"])].shape[0]/train_df[train_df.c!=0].shape[0]}')
  # print(f'Porcentagem classificadas corretamente das totais: {train_df[(train_df.c!=0) & (train_df.c==train_df["ap.age"])].shape[0]/train_df.shape[0]}')

  test_matrix = vectorizer.transform(test_df["Clean_text"].to_list())
  test_matrix = test_matrix.toarray()

  tf_idf_test = pd.DataFrame(data=test_matrix, columns=vocab)

  test_df.reset_index(inplace=True)

  test_df['glex1']=0
  test_df['glex2']=0
  test_df['glex3']=0

  for index, row in test_df.iterrows():
      sum = 0
      text = row["Clean_text"].split()
      for w in text:
        if w in coef1.index.tolist():
          sum += coef1[0][w]*tf_idf_test[w][index]
      sum = sum + svc.intercept_[0]
      test_df[f'glex1'][index] = sum

  for index, row in test_df.iterrows():
      sum = 0
      text = row["Clean_text"].split()
      for w in text:
        if w in coef2.index.tolist():
          sum += coef2[1][w]*tf_idf_test[w][index]
      sum = sum + svc.intercept_[1]
      test_df[f'glex2'][index] = sum

  for index, row in test_df.iterrows():
      sum = 0
      text = row["Clean_text"].split()
      for w in text:
        if w in coef3.index.tolist():
          sum += coef3[2][w]*tf_idf_test[w][index]
      sum = sum + svc.intercept_[2]
      test_df[f'glex3'][index] = sum

  test_df['c'] = 0

  for index, row in test_df.iterrows():
    if row['glex1'] > 0 and row['glex2'] > 0 and row['glex3'] > 0:
      test_df['c'][index] = 1
    elif row['glex1'] < 0 and row['glex2'] > 0 and row['glex3'] > 0:
      test_df['c'][index] = 2
    elif row['glex1'] < 0 and row['glex2'] < 0 and row['glex3'] < 0:
      test_df['c'][index] = 3

  test_df['ap.age'] = test_df['ap.age'].astype('int64')

  #print(f"Classificação por dicionário: {test_df[test_df.c!=0].shape[0]} instâncias")
  #print(f'Porcentagem classificadas: {test_df[test_df.c!=0].shape[0]/test_df.shape[0]}')
  #print(f'Classificadas corretamente: {test_df[(test_df.c!=0) & (test_df.c==test_df["ap.age"])].shape[0]/test_df[test_df.c!=0].shape[0]}')

  #print(f"Classificação tradicional: {test_df[test_df.c==0].shape[0]} instâncias")

  age_test_dict = test_df[test_df.c!=0]["ap.age"].to_list()
  pred_age_dict = test_df[test_df.c!=0]["c"].to_list()

  tfvec = TfidfVectorizer(max_features = 3000)
  tfvec.fit(train_df['Clean_text'].to_list())
  tdf_train = tfvec.transform(train_df['Clean_text'].to_list()).toarray().tolist()

  age_train = train_df["ap.age"].to_list()

  logisticRegr = LogisticRegression(penalty=age_param["k_"+str(i)]["params"]["penalty"], C=age_param["k_"+str(i)]["params"]["C"], solver='liblinear')
  logisticRegr.fit(tdf_train, age_train)
  
  test_df_dict = test_df[test_df.c!=0]
  tdf_test_dict = tfvec.transform(test_df_dict['Clean_text'].to_list()).toarray().tolist()

  pred_age_trad_dict=logisticRegr.predict(tdf_test_dict)

  print(f"F1 (trad-dict): {metrics.f1_score(age_test_dict, pred_age_trad_dict, average='macro')}")
  print(f"F1 (dict): {metrics.f1_score(age_test_dict, pred_age_dict, average='macro')}")

  test_df_noclass_dict = test_df[test_df.c==0]
  age_test_noclass_dict = test_df_noclass_dict["ap.age"].to_list()
  tdf_test_noclass_dict = tfvec.transform(test_df_noclass_dict['Clean_text'].to_list()).toarray().tolist()

  pred_age_noclass_dict=logisticRegr.predict(tdf_test_noclass_dict) 

  class_total = age_test_dict+age_test_noclass_dict
  pred_total = pred_age_dict+pred_age_noclass_dict.tolist()

  print(f"RESULTADO: {metrics.f1_score(class_total, pred_total, average='macro')}")
  print("-----------------------------------------------------------------------")

F1 (trad-dict): 0.4738134964775677
F1 (dict): 0.5107239376689092
RESULTADO: 0.48081081081081084
-----------------------------------------------------------------------
F1 (trad-dict): 0.5223706229673835
F1 (dict): 0.5186495650891936
RESULTADO: 0.5073610599926389
-----------------------------------------------------------------------
F1 (trad-dict): 0.491027926322044
F1 (dict): 0.565826330532213
RESULTADO: 0.5679671205986995
-----------------------------------------------------------------------
F1 (trad-dict): 0.40803621958121106
F1 (dict): 0.3924963924963925
RESULTADO: 0.4593563815531779
-----------------------------------------------------------------------
F1 (trad-dict): 0.451739618406285
F1 (dict): 0.4791666666666667
RESULTADO: 0.4626180836707152
-----------------------------------------------------------------------
F1 (trad-dict): 0.1904761904761905
F1 (dict): 0.46969696969696967
RESULTADO: 0.407392253136934
-----------------------------------------------------------------------